In [1]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
! conda install -c rdkit rdkit -y

--2021-12-04 08:58:40--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.1’

Miniconda3-py37_4.8 100%[===================>]  81.12M   152MB/s    in 0.5s    

2021-12-04 08:58:40 (152 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.1’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: \ | / - \ | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::jpeg==9d=h7f8727e_0
  - defaults/linux-64::libuuid==1.0.3=h7f8727e_2
  - defaults/linux-64::numpy-base==1.21.2=py37h79a1101_0
  - defaults/linux-64::mk

In [2]:
# 导入包
import torch
import torch.optim as optim
from torch import nn as nn
import torch.nn.functional as F
from torch import sigmoid
from torch.autograd import Variable
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

In [3]:
data = pd.read_csv('molnet_bace.csv')
data_sub = data[['mol', 'pIC50']]
train_data_x, test_data_x, train_y, test_y = train_test_split(data_sub['mol'], data_sub['pIC50'], test_size = 0.2, random_state = 1 )

# data process

In [4]:
train_mols = [Chem.MolFromSmiles(smi) for smi in train_data_x]
train_morgan_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in train_mols]
train_morgan_fps_array = np.asarray(train_morgan_fps, dtype=float)
train_x_tensor = torch.from_numpy(train_morgan_fps_array)
train_y_tensor = torch.from_numpy(np.array(train_y))

In [5]:
test_mols = [Chem.MolFromSmiles(smi) for smi in test_data_x]
test_morgan_fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in test_mols]
test_morgan_fps_array = np.asarray(test_morgan_fps, dtype=float)
test_x_tensor = torch.from_numpy(test_morgan_fps_array)
test_y_tensor = torch.from_numpy(np.array(test_y))

# model

In [14]:
class QSAR_mlp(nn.Module):  # 继承torch的module，即可以直接拥有 torch.nn.Module 的属性和方法
    def __init__(self):
        super(QSAR_mlp, self).__init__()
        self.fc1 = nn.Linear(2048, 524) # 对输入数据进行线性转换
        self.fc2 = nn.Linear(524, 10)
        self.fc3 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 1) 
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        output = self.fc4(h3)
        return output

In [27]:
epochs = 200
criterion = nn.MSELoss()

model = QSAR_mlp()
losses = []
optimizer = optim.Adam(model.parameters(), lr=0.005)
for epoch in range(epochs):
    data, target = Variable(train_x_tensor).float(), Variable(train_y_tensor).float()
    optimizer.zero_grad()
    y_pred = model(data)
    #loss = F.cross_entropy(y_pred, target)
    loss = criterion(y_pred.reshape(-1), target)
    losses.append(loss.item())
    #if epoch % 10 == 0:
    print("Loss {}: {}".format(epoch, loss.item()))
    loss.backward()
    optimizer.step()

Loss 0: 44.72142791748047
Loss 1: 43.451942443847656
Loss 2: 43.170188903808594
Loss 3: 41.47317123413086
Loss 4: 39.67295837402344
Loss 5: 37.3197135925293
Loss 6: 34.359779357910156
Loss 7: 30.7624454498291
Loss 8: 26.536235809326172
Loss 9: 21.755640029907227
Loss 10: 16.59102439880371
Loss 11: 11.356934547424316
Loss 12: 6.56884241104126
Loss 13: 2.9839110374450684
Loss 14: 1.5500566959381104
Loss 15: 2.9045379161834717
Loss 16: 6.024860382080078
Loss 17: 8.271947860717773
Loss 18: 8.303936004638672
Loss 19: 6.645246982574463
Loss 20: 4.399960517883301
Loss 21: 2.4590234756469727
Loss 22: 1.265616774559021
Loss 23: 0.8595501184463501
Loss 24: 1.0401180982589722
Loss 25: 1.5282920598983765
Loss 26: 2.0753915309906006
Loss 27: 2.5092005729675293
Loss 28: 2.73685622215271
Loss 29: 2.730515718460083
Loss 30: 2.5103871822357178
Loss 31: 2.1305744647979736
Loss 32: 1.6675013303756714
Loss 33: 1.208496332168579
Loss 34: 0.838031530380249
Loss 35: 0.6205554604530334
Loss 36: 0.582302927970

In [28]:
pred_train_y = model(Variable(train_x_tensor).float()).reshape(-1).tolist()
pred_test_y = model(Variable(test_x_tensor).float()).reshape(-1).tolist()

# evaluate model

In [29]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [35]:
nn_test_mae = round(mean_absolute_error(test_y, pred_test_y), 2)
nn_test_mse = round(mean_squared_error(test_y, pred_test_y), 2)
nn_test_mape = round(mean_absolute_percentage_error(test_y, pred_test_y), 2)
nn_test_r2 = round(r2_score(test_y, pred_test_y), 2)


nn_train_mae = round(mean_absolute_error(train_y, pred_train_y), 2)
nn_train_mse = round(mean_squared_error(train_y, pred_train_y), 2)
nn_train_mape = round(mean_absolute_percentage_error(train_y, pred_train_y), 2)
nn_train_r2 = round(r2_score(train_y, pred_train_y), 2)

In [38]:
nn_perf = {'mae':[nn_train_mae, nn_test_mae],
            'mse':[nn_train_mse, nn_test_mse],
            'mape':[nn_train_mape, nn_test_mape],
            'r2':[nn_train_r2, nn_test_r2]}

In [39]:
pd.DataFrame(nn_perf, index=['train', 'test'])

,mae,mse,mape,r2
train,0.18,0.08,0.03,0.96
test,0.50,0.48,0.08,0.72
